In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
Medicine_Meta_Llama_LoRA = "MihirKapile/Hindi-Medicine-Meta-Llama-3.2-3B-LoRA"
Law_Meta_Llama_LoRA = "MihirKapile/Hindi-Law-Meta-Llama-3.2-3B-LoRA"

In [4]:
medicine_model, medicine_tokenizer = FastLanguageModel.from_pretrained(
    model_name= Medicine_Meta_Llama_LoRA,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Enable faster inference
FastLanguageModel.for_inference(medicine_model)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Unsloth 2024.11.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [5]:
law_model, law_tokenizer = FastLanguageModel.from_pretrained(
    model_name= Law_Meta_Llama_LoRA,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Enable faster inference
FastLanguageModel.for_inference(law_model)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [6]:
# Alpaca-style prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following question in Hindi:

### Input:
{}

### Response:
{}"""

In [7]:
def parse_response(output):
    # Ensure the output is a string
    if isinstance(output, list):
        output = output[0]

    # Define the start and end markers
    start_marker = "### Response:\n"
    end_marker = "<|eot_id|>"

    # Extract the response between the markers
    if start_marker in output and end_marker in output:
        start_index = output.index(start_marker) + len(start_marker)
        end_index = output.index(end_marker)
        response = output[start_index:end_index].strip()
        return response
    return None

In [8]:
def get_Medicine_answers(question, alpaca_prompt=alpaca_prompt, model=medicine_model, tokenizer=medicine_tokenizer):

    # Prepare the input using the tokenizer
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                question,  # Question is passed here
                "",        # Output - leave this blank for generation
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, temperature=0.1, use_cache = True)
    tokenizer.batch_decode(outputs)
    return parse_response(tokenizer.batch_decode(outputs))

In [10]:
def get_Law_answers(question, alpaca_prompt=alpaca_prompt, model=law_model, tokenizer=law_tokenizer):

    # Prepare the input using the tokenizer
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                question,  # Question is passed here
                "",        # Output - leave this blank for generation
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, temperature=0.1, use_cache = True)
    tokenizer.batch_decode(outputs)
    return parse_response(tokenizer.batch_decode(outputs))

In [11]:
res = get_Law_answers("सिविल कानून और फौजदारी कानून में क्या अंतर है?")
print(res)

सिविल कानून में व्यक्तिगत संपत्ति के संबंध में नियम होते हैं जबकि फौजदारी कानून में अपराध के संबंध में नियम होते हैं।


In [12]:
medicine_tool={
    "type": "function",
    "function": {
        "name": "get_Medicine_answers",
        "description": "Provides medical information and answers to health-related queries.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The medical question or topic needing an answer."
                }
            },
            "required": ["question"]
        }
    }
}

law_tool={
    "type": "function",
    "function": {
        "name": "get_Law_answers",
        "description": "Provides legal information and answers to law-related queries.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The legal question or topic needing an answer."
                }
            },
            "required": ["question"]
        }
    }
}

In [13]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.3 MB/s eta 0:00:00


In [15]:
import os
from groq import Groq
from google.colab import userdata
import json
groq_api_key = userdata.get('GROQ_API_KEY')

In [16]:
client = Groq(api_key = groq_api_key)

In [17]:
# Initialize an empty history list
history = []

# Loop for chat
while True:
    # Get user input
    user_input = input("You: ")

    # Exit the loop if the user types 'exit'
    if user_input.lower() == "exit":
        break

    # Append the user's input to the history
    history.append({"role": "user", "content": user_input})

    # Make the chat completion request with the current history
    chat_completion = client.chat.completions.create(
        messages=history,
        model="llama-3.2-90b-vision-preview",
    )

    # Get the assistant's response
    assistant_response = chat_completion.choices[0].message.content
    print(f"Assistant: {assistant_response}")

    # Append the assistant's response to the history
    history.append({"role": "assistant", "content": assistant_response})

You: Hello
Assistant: Hello. It's nice to meet you. Is there something I can help you with, or would you like to chat?
You: who am I?
Assistant: I don't have any information about your personal identity, as our conversation just started. If you'd like to share something about yourself, I'm here to listen. Otherwise, I'll just consider you a friendly conversation partner.
You: whats my name?
Assistant: I don't know your name. You haven't shared it with me, and I don't have any way to access that information. If you'd like to share your name, I'd be happy to use it to address you in our conversation.
You: so my name is Mihir and i am testing LLMs
Assistant: Nice to meet you, Mihir. It's great that you're testing LLMs (Large Language Models) like me. I'm here to learn and improve from interactions like ours. Go ahead and test my capabilities, ask me questions, or challenge me in any way you'd like. I'll do my best to respond accurately and helpfully.

What specific aspects of LLMs are you

KeyboardInterrupt: Interrupted by user

In [18]:
pip install rich

In [19]:
from rich import print

In [21]:
history = [
    {
        "role": "system",
        "content": "You are a chat assistant with specialized tools for answering questions in the fields of medicine and law, and you respond only in Hindi."
    }
]

# Define the available tools (i.e. functions) for our model to use
tools = [medicine_tool, law_tool]

# Selecting the model with tool capabilities
MODEL = "llama-3.2-90b-vision-preview"

while True:
    # Get user input
    user_input = input("You: ")

    # Exit the loop if the user types 'exit'
    if user_input.lower() == "exit":
        print("Ending conversation. Goodbye!")
        break

    # Add the user's input to the conversation history
    history.append({"role": "user", "content": user_input})

    # Make the API call to Groq
    response = client.chat.completions.create(
        model=MODEL,
        messages=history,
        stream=False,
        tools=tools,
        tool_choice="auto",
        temperature=0,
        max_tokens=4096
    )

    # Extract the response message and any tool call responses
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    history.append({"role": "assistant", "content": response_message.content})

    # Print the assistant's response
    if response_message.content is not None:
      print(f"[green]Assistant: {response_message.content}[/green]")

    # If the model requested tool calls, handle them
    if tool_calls:
        # Define the available functions that can be called by the LLM
        available_functions = {
            "get_Medicine_answers": get_Medicine_answers,
            "get_Law_answers": get_Law_answers,
        }

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)
                # Call the tool and get the response
                function_response = function_to_call(
                    question=function_args.get("question")
                )
                # Add the tool response to the conversation history
                history.append({
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                    "tool_call_id": tool_call.id  # Associate response with the tool call
                })

                # Display the tool's response
                print(f"[blue][{function_name} Tool]: {function_response}[/blue]")

        # Re-run the model with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            messages=history
        )

        # Print the final response from the assistant
        final_response = second_response.choices[0].message.content
        print(f"[green]Assistant: {final_response}[/green]")
        # Update the history with the assistant's final response
        history.append({"role": "assistant", "content": final_response})

You: namaskar


Assistant: नमस्कार, मैं आपकी सहायता के लिए यहाँ हूँ। कृपया अपना प्रश्न पूछें ताकि मैं आपको चिकित्सा या कानून संबंधी जानकारी प्रदान कर सकूँ।

You: सिविल कानून और फौजदारी कानून में क्या अंतर है?


: सिविल कानून में प्रतिवादी के अधिकारों को बढ़ावा दिया जाता है जबकि फौजदारी कानून में प्रतिवादी के अधिकारों को कम किया जाता है।

Assistant: सिविल कानून और फौजदारी कानून दो अलग-अलग प्रकार के कानून हैं जो विभिन्न प्रकार के मामलों को संभालते हैं।

सिविल कानून:

सिविल कानून व्यक्तिगत और सामाजिक संबंधों को नियंत्रित करता है, जैसे कि संपत्ति के अधिकार, अनुबंध, विवाह, तलाक, और परिवार के मामले। यह कानून व्यक्तियों के बीच विवादों को
सुलझाने के लिए बनाया गया है, जैसे कि संपत्ति के विवाद, अनुबंध के उल्लंघन, और व्यक्तिगत चोटों के मामले।

फौजदारी कानून:

फौजदारी कानून अपराधों को परिभाषित करता है और उनके लिए सजा का प्रावधान करता है। यह कानून समाज की सुरक्षा और शांति को बनाए रखने के लिए बनाया गया है। फौजदारी कानून के
तहत, अपराधों को दो श्रेणियों में बांटा जाता है: गंभीर अपराध (जैसे कि हत्या, डकैती, और बलात्कार) और कम गंभीर अपराध (जैसे कि चोरी, धोखाधड़ी, और सार्वजनिक स्थानों पर
अश्लीलता)।

अंतर:

सिविल कानून और फौजदारी कानून के बीच मुख्य अंतर यह है कि सिविल कानून व्यक्तिगत और सामाजिक संबंधों को नियंत्रित करता है, जबकि फौजदारी कानून अपराधों को परिभाषित करता है
और उनके लिए सजा का प्रावधान करता है। सिविल कानून के तहत, व्यक्तियों को अपने अधिकारों की रक्षा के लिए अदालत में जाने का अधिकार होता है, जबकि फौजदारी कानून के तहत,
राज्य अपराधों की जांच और अभियोजन करता है।

You: end


Assistant: अलविदा, मुझे आशा है कि मैं आपकी सहायता कर सका। यदि आपके पास और प्रश्न हैं, तो मुझसे कभी भी पूछें।

You: exit


Ending conversation. Goodbye!